<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/031_Function_Calling_Capabilities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## 📘 LUsing Function Calling Capabilities with LLMs

In this lesson, we explore **how LLMs can be integrated with tools** using function calling — a structured and robust way to turn free-form user requests into executable actions.

### 🧠 Key Takeaways

* **Function calling enables structured tool usage.**
  Instead of trying to prompt the model to generate JSON (which is unreliable), we use OpenAI’s function calling API, which guarantees structured output using predefined JSON schemas.

* **The LLM decides when to use a tool.**
  You provide the tools and schema, and the model decides whether to return a function call (with arguments) or a regular assistant message.

* **No need for complex prompt engineering.**
  Function calling APIs separate *decision logic* (what to do) from *formatting logic* (how to do it). You only define the tools and their expected inputs—OpenAI handles the rest.

* **The LLM outputs a `tool_calls` array** (if it chooses a tool), with perfectly structured JSON:

  ```json
  {
    "tool_calls": [
      {
        "function": {
          "name": "read_file",
          "arguments": "{ \"filename\": \"example.txt\" }"
        }
      }
    ]
  }
  ```

* **Your code receives this structured result and routes it to the matching function** using Python (e.g., `tool_router()` or a function registry).





## ✅ Key Benefits of Function Calling APIs

- **Eliminates prompt engineering for structured responses**  
  No need to force the model to output JSON manually.

- **Uses standardized JSON Schema**  
  The same format used in API documentation applies seamlessly to AI interactions.

- **Allows mixed text and tool execution**  
  The model can decide whether a tool is necessary or provide a natural language response.

- **Simplifies parsing logic**  
  Instead of handling inconsistent outputs, developers only check for `tool_calls` in the response.

- **Guarantees syntactically correct arguments**  
  The model automatically ensures arguments match the expected parameter format.

---

## 🧩 Conclusion

Function calling APIs significantly improve the reliability of AI-agent interactions by enforcing structured execution. By defining tools with JSON Schema and letting the model determine when to use them, we build a more predictable and maintainable AI interface.


## 🎯 Notebook Goals

In this notebook, we will:

1. Reiterate and apply the 8-step process for enabling function calling.
2. Review the anatomy of a tool: from Python function → JSON Schema → OpenAI tool definition.
3. Use OpenAI’s built-in `tool_calls` parsing to extract which function the LLM wants to use.
4. Route and execute tool calls dynamically.
5. Understand the benefits and limitations of this approach.





## 🧠 8-Step Process for Enabling Function Calling in Agents

This process outlines how to go from user input to executing real code with structured tool support using OpenAI's function calling features.

---

### 1. **Define Your Python Functions (Tools)**
These are the real backend functions that execute logic — reading files, searching directories, analyzing data, etc.

```python
def read_file(filename): ...
def list_files(): ...
def search_file_names(keyword, case_sensitive=False): ...
````

---

### 2. **Describe Each Tool Using JSON Schema**

Wrap each function in a JSON-compliant schema that defines:

* Tool name
* Description
* Parameters (`type`, `properties`, `required`)

This schema tells the LLM what the tool does and what inputs it requires.

---

### 3. **Combine Tool Definitions into a `tools` List**

Format tools according to the OpenAI function-calling spec:

```python
tools = [
    {
        "type": "function",
        "function": list_files_tool
    },
    ...
]
```

---

### 4. **Send Tools to the LLM in the API Call**

Pass your `tools` list to the `chat.completions.create()` method.

```python
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    tools=tools,
    tool_choice="auto"
)
```

---

### 5. **LLM Decides: Tool Call or Plain Text**

The LLM returns either:

* Natural language response (`choice.content`)
* OR a structured tool call block (`choice.tool_calls`)

---

### 6. **Parse the Tool Call Block**

If a tool is selected, extract:

* `tool_name = choice.tool_calls[0].function.name`
* `args = json.loads(choice.tool_calls[0].function.arguments)`

---

### 7. **Route to the Correct Python Function**

Use a tool router that maps `tool_name` to your backend logic:

```python
def tool_router(tool_name, args): ...
```

---

### 8. **Execute the Tool and Return Result**

Once the tool is run, you can:

* Return the result back to the user
* Feed it back to the LLM for further reasoning or dialogue

```python
result = tool_router(tool_name, args)
```

---

## ✅ Summary

This structured flow turns the LLM into an intelligent **decision-maker** that:

* Interprets the user’s intent
* Matches it to the correct tool
* Ensures inputs are valid
* And executes real actions

With minimal parsing and robust logic, this model-driven architecture brings reliability and flexibility to agent design.




## 🛠️ Part 1: Python Tool Functions

We'll define two simple Python tools:

### 1. `get_stock_price(ticker: str)`

Simulates getting the current price for a given stock ticker.

### 2. `search_stock_news(ticker: str)`

Simulates returning a recent news headline for a stock.

(These are mock tools for now, but later we can plug in real APIs like `yfinance` or web search.)


In [1]:
import random

# 🟦 Simulated stock price tool
def get_stock_price(ticker: str) -> str:
    mock_price = round(random.uniform(100, 500), 2)
    return f"{ticker.upper()} is currently trading at ${mock_price}."

# 🟨 Simulated news search tool
def search_stock_news(ticker: str) -> str:
    sample_news = {
        "AAPL": "Apple unveils new AI-powered chip at WWDC.",
        "TSLA": "Tesla's Q2 deliveries beat expectations, stock jumps.",
        "NVDA": "NVIDIA partners with OpenAI for next-gen GPU architecture.",
    }
    return sample_news.get(ticker.upper(), f"No recent news found for {ticker.upper()}.")

## 🧰 Part 2: Define Tool Schemas

In [19]:
# 🛠️ Tool schema for get_stock_price
get_stock_price_tool = {
    "type": "function",
    "function": {
        "name": "get_stock_price",
        "description": "Gets the current price of a given stock ticker symbol.",
        "parameters": {
            "type": "object",
            "properties": {
                "ticker": {
                    "type": "string",
                    "description": "The stock ticker symbol (e.g., AAPL, TSLA)."
                }
            },
            "required": ["ticker"]
        }
    }
}

# 🛠️ Tool schema for search_stock_news
search_stock_news_tool = {
    "type": "function",
    "function": {
        "name": "search_stock_news",
        "description": "Searches for the most recent news about a given stock ticker.",
        "parameters": {
            "type": "object",
            "properties": {
                "ticker": {
                    "type": "string",
                    "description": "The stock ticker symbol (e.g., AAPL, TSLA)."
                }
            },
            "required": ["ticker"]
        }
    }
}

# 🧰 Master tools list
tools = [get_stock_price_tool, search_stock_news_tool]


Now onto **🤖 Part 3: Agent Response Generator**

We'll create a function that:

1. Accepts the **user input**
2. Builds a chat message history
3. Uses the OpenAI API to generate a response
4. Returns either:

   * a natural language reply
   * or a **tool call** if the model chooses to use a function

---

## 🤖 Part 3: Generate Agent Response



In [4]:
%pip install -qU dotenv openai

In [20]:
import json
from openai import OpenAI
from dotenv import load_dotenv
import os

# ✅ Load API key
load_dotenv("/content/API_KEYS.env")
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# 🤖 Agent response generator
def generate_agent_response(user_input):
    messages = [
        {
            "role": "system",
            "content": "You are a financial assistant who can retrieve real-time stock prices and search recent news using tools. Be concise and helpful."
        },
        {"role": "user", "content": user_input}
    ]

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        tools=tools,              # 🧰 Our tool schemas
        tool_choice="auto"        # 🔍 Let model decide
    )

    choice = response.choices[0].message

    if choice.tool_calls:
        # 🛠️ Model wants to use a tool
        tool_call = choice.tool_calls[0]
        return {"type": "tool", "tool_call": tool_call}
    else:
        # 💬 Regular message
        return {"type": "text", "content": choice.content}


## 🔧 Part 4: Define the Tool Functions
These are the actual Python implementations that the agent will call once the LLM chooses a tool and returns the necessary arguments.

In [21]:
# 🔧 Tool 1: Get stock price (mocked)
def get_stock_price(ticker):
    # In real use case, you'd call an API like Yahoo Finance or Alpha Vantage
    mock_prices = {
        "AAPL": 187.65,
        "GOOGL": 2765.21,
        "MSFT": 412.88,
        "TSLA": 253.12
    }
    price = mock_prices.get(ticker.upper(), "Unknown Ticker")
    return f"The current price of {ticker.upper()} is ${price}"

# 🔧 Tool 2: Search finance news (mocked)
def search_news(ticker, limit=3):
    mock_articles = {
        "apple": [
            "Apple announces record quarterly earnings.",
            "New iPhone 16 rumored to launch this fall.",
            "Apple stock surges on AI push."
        ],
        "tesla": [
            "Tesla reports delivery beat for Q2.",
            "Elon Musk teases Robotaxi prototype reveal.",
            "Tesla stock jumps 12% after earnings."
        ]
    }
    articles = mock_articles.get(ticker.lower(), ["No news found."])
    return "\n".join(articles[:limit])


## Part 5: Tool Router – Executing the Chosen Tool

In [22]:
def tool_router(tool_name, args):
    if tool_name == "get_stock_price":
        return get_stock_price(args["ticker"])
    elif tool_name == "search_stock_news":
        return search_news(
            ticker=args["ticker"],
            limit=args.get("limit", 3)
        )
    else:
        return f"❌ Unknown tool: {tool_name}"


## Part 6: Handle Tool Call

In [23]:
def handle_tool_call_debug(choice):
    if choice["type"] == "tool":
        tool_name = choice["tool_call"].function.name
        args = json.loads(choice["tool_call"].function.arguments)
        result = tool_router(tool_name, args)
        print(f"🛠️ Tool Used: {tool_name}")
        print(f"📤 Args: {args}")
        print(f"📥 Result:\n{result}")
    elif choice["type"] == "text":
        print(f"💬 Assistant Response:\n{choice['content']}")
    else:
        print("❓ Unexpected response type.")

## ⚙️ Part 7: Run

In [26]:
user_input = "What's the current stock price of AAPL?"
choice = generate_agent_response(user_input)
handle_tool_call_debug(choice)

🛠️ Tool Used: get_stock_price
📤 Args: {'ticker': 'AAPL'}
📥 Result:
The current price of AAPL is $187.65


In [28]:
def handle_tool_call_conversational(tool_call):
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)
    result = tool_router(name, args)

    # 👇 This is the missing step — add the assistant's tool call message
    assistant_message = {
        "role": "assistant",
        "tool_calls": [tool_call]  # 👈 This is required
    }

    follow_up = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a financial assistant."},
            {"role": "user", "content": f"What's the current stock price of {args['ticker']}?"},
            assistant_message,
            {
                "role": "tool",
                "tool_call_id": tool_call.id,
                "name": name,
                "content": result
            }
        ]
    )

    return follow_up.choices[0].message.content

tool_call = choice["tool_call"]
response = handle_tool_call_conversational(tool_call)
print(response)


The current stock price of AAPL (Apple Inc.) is $187.65.


**Mocking up an agent is *absolutely* the best first step** before implementing real API calls. Here's why it's a smart and strategic move:

---

### ✅ Why Mocking Agents First is a Great Idea

#### 1. **Clarifies agent behavior**

* You define *what* the agent should do before worrying about *how*.
* It helps you test conversation flow, tool orchestration, and logic independently of real-world errors or API quirks.

#### 2. **Zero risk and low cost**

* No real API keys, credentials, or rate limits to worry about.
* You avoid being billed while testing logic and prompt structure.

#### 3. **Enables fast iteration**

* You can focus on refining tool schemas, tool selection, routing, and conversation design.
* No waiting on slow or flaky APIs during debugging.

#### 4. **Improves prompt and tool schema quality**

* You'll catch edge cases where the LLM picks the wrong tool or misunderstands parameters — and fix those early.
* Once you swap in real APIs, the behavior should already be stable.

#### 5. **Easy to switch to production**

* When you're ready, replace mock return values with real API calls — the rest of the system stays the same.
* Your router, agent loop, tool schema, and error handling logic are already done.

---

### 🛠️ Example Workflow

| Step | Description                                                                          |
| ---- | ------------------------------------------------------------------------------------ |
| 1️⃣  | Build the mock tool function (`def get_weather(city): return "It's sunny in Miami"`) |
| 2️⃣  | Define the tool schema using JSON Schema                                             |
| 3️⃣  | Add it to your agent and test interaction                                            |
| 4️⃣  | Refine prompt, tool call, and return handling                                        |
| 5️⃣  | Swap mock function with real API call when stable                                    |

---

### 🔄 Mock → Real API Transition

For each mock tool, you’ll eventually:

* Import a requests or API client library
* Replace mock return value with real data fetch + parsing
* Keep the tool signature and schema unchanged

---

**TL;DR**: Mocking gives you clean separation between *agent logic* and *data plumbing*, making everything more reliable, scalable, and easier to debug.





### 🧠 Core Lessons to Remember

#### 1. **LLMs can *decide* when to use tools**

* `tool_choice="auto"` lets the LLM evaluate whether a tool is appropriate.
* Your job is to clearly define tool schemas and let the LLM orchestrate execution.

#### 2. **Tool schemas must match OpenAI’s JSON Schema format**

* Every tool must be declared using:

  * `"type": "function"`
  * `"function"` key with name, description, parameters
* Inputs must be JSON-serializable.

#### 3. **Tool outputs must be fed back into the conversation**

* The LLM won’t “see” a tool result unless you return it using the special `role: "tool"` message.
* It must be paired with the *original assistant message* that triggered the tool via `tool_calls`.

#### 4. **You're not replacing the LLM — you're empowering it**

* The LLM is your reasoning engine.
* Your tools are just "muscles" it can use for precise tasks like file access, retrieval, API calls, etc.

---

### 💡 Pro Tips

* **Log every tool call.** It’s super helpful for debugging and transparency.
* **Always test tools independently.** If a tool fails, isolate and fix before rerunning the agent loop.
* **Use clear tool descriptions.** That helps the LLM know *when* it should call each tool.
* **Build small at first.** Then compose multiple tools for more complex agents later.

---

### 🧭 What's Next?

You're now ready for:

* ✅ Memory integration (so the agent can build context over time)
* ✅ Multi-step planning (chaining tools based on intermediate results)
* ✅ Tool routing with decorators (to cleanly register tools)


